## 2.1 Experiment with Bigger Neural Network 
- Adding Layer 
- Change Activation


⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-Image-Classification/blob/main/Pertemuan%202/2.1%20bigger_network.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="resource/cl-connect-gpu.png" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="resource/cl-connect-gpu-success.png" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

- All code remain same with <font color="orange">Pertemuan 1</font>, but different in designing model part.

In [ ]:
!pip install gdown

import os
import cv2
import gdown
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import transforms

from IPython import display

# clear output cell
display.clear_output()

print(f"torch : {torch.__version__}")
print(f"torch vision : {torchvision.__version__}")

- Download MNIST Dataset

In [ ]:
DATASET_NAME = 'MNIST' # the dataset name
DATASET_NUM_CLASS = 10 # number of class in dataset

In [ ]:
# default using gdrive_id Dataset `mnist_dataset.zip` (1-FfwJrllyHofQwIbMb_IxAkxnfMGSFmR)
gdrive_id = '1-FfwJrllyHofQwIbMb_IxAkxnfMGSFmR' # <-----  ⚠️⚠️⚠️ USE YOUR OWN GDrive ID FOR CUSTOM DATASET ⚠️⚠️⚠️

# download zip from GDrive
url = f'https://drive.google.com/uc?id={gdrive_id}'
gdown.download(url, DATASET_NAME + ".zip", quiet=False)

# unzip dataset
!unzip {DATASET_NAME}.zip -d {DATASET_NAME}

# clear output cell
display.clear_output()

- Load MNIST Dataset

In [ ]:
# Define Custom Dataset class
# it's just helper to load image dataset using OpenCV and convert to pytorch tensor
# also doing a label encoding using one-hot encoding
class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.image_files = sorted([file for file in os.listdir(root_dir) if file.lower().endswith('.png')])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Read image from corresponding .png file
        image_path = os.path.join(self.root_dir, self.image_files[idx])
        image = cv2.imread(image_path)  # Load image using OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert BGR to GRAY
        image = torch.from_numpy(image).to(torch.float32)  # Convert NumPy array to PyTorch tensor
        image = torch.flatten(image) # flatten image from 2D Tensor (28x28) into a 1D tensor (784)

        # Read label from corresponding .txt file
        label_path = os.path.splitext(image_path)[0] + ".txt"
        with open(label_path, 'r') as label_file:
            label = int(label_file.read().strip())  # Assuming labels are integers

        # Apply one-hot encoding into label
        labels_tensor = torch.tensor(label)
        one_hot_encoded = F.one_hot(labels_tensor, num_classes=DATASET_NUM_CLASS).to(torch.float32)

        return image, one_hot_encoded



# instantiate dataset
# in here the image dataset is not loaded yet
# we only read all image files names in fataset folder
all_train_dataset = CustomDataset(root_dir=f'{DATASET_NAME}/dataset/train')
test_dataset = CustomDataset(root_dir=f'{DATASET_NAME}/dataset/test')

In [ ]:
print(f"All Train Dataset : {len(all_train_dataset)} data")
print(f"Test Dataset : {len(test_dataset)} data")

In [ ]:
# Split 'all_train_dataset' into 'train' and 'validation' set using `random_split()` function
train_dataset, validation_dataset = random_split(all_train_dataset, [50000, 10000])

print(f"Train Dataset : {len(train_dataset)} data")
print(f"Validation Dataset : {len(validation_dataset)} data")

In [ ]:
# Create data loaders
BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### 2.1.1 Experiment Adding more Layer to the Model
- To improve the recognition accuracy we will <font color="orange">add more layers</font> to the neural network.<br><br>
<img src="resource/NN_arch_1.png" width="700px"><br><br>
- We keep softmax as the activation function on the last layer because that is what works best for classification. 
- On <font color="orange">hidden layers</font> however we will use the most classical activation function: the <font color="orange">sigmoid</font>:<br>
<img src="resource/NN_Sigmoid.png" width="400px"><a id="model-definition"></a>

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define a modified model with additional layers
model = nn.Sequential(
    nn.Linear(in_features=784, out_features=200),   # 1st hidden layer
    nn.Sigmoid(),                                   # Sigmoid Activation function
    nn.Linear(in_features=200, out_features=60),    # 2nd hidden layer
    nn.Sigmoid(),                                   # Sigmoid Activation function
    nn.Linear(in_features=60, out_features=10),     # Output layer
    nn.LogSoftmax(dim=1)                            # Log probabilities for classification
).to(device)

In [ ]:
# setup optimizer, loss function & metric
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_function = nn.CrossEntropyLoss()

- To run training process, we can use the following code

In [ ]:
!pip install tqdm

from tqdm import tqdm

In [ ]:
def train(model, train_loader, optimizer, loss_function):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Add progress bar for training loop
    progress_bar = tqdm(train_loader, desc='Training', leave=False)

    for inputs, labels in progress_bar:
        inputs = inputs.to(device) # move inputs to device
        labels = labels.to(device) # move labels to device

        # resets the gradients of all the model's parameters before the backward pass
        optimizer.zero_grad()
        # pass input tensor to model
        outputs = model(inputs)
        # calc loss value
        loss = loss_function(outputs, labels)
        # computes the gradient of the loss with respect to each parameter in model
        loss.backward()
        # adjust model parameter
        optimizer.step()
        # sum loss value
        running_loss += loss.item()

        # Calculate correct & total prediction
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels.argmax(1)).sum().item()
        total_predictions += labels.size(0)

        # Update progress bar description with current loss
        progress_bar.set_postfix(loss=loss.item())

    # Calculate average training loss
    average_train_loss = running_loss / len(train_loader.dataset)
    # Calculate training accuracy
    train_accuracy = correct_predictions / total_predictions
    return average_train_loss, train_accuracy

def validate(model, val_loader, loss_function):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    # Add progress bar for validation loop
    progress_bar = tqdm(val_loader, desc='Validating', leave=False)

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs = inputs.to(device) # move inputs to device
            labels = labels.to(device) # move labels to device

            # pass input tensor to model
            outputs = model(inputs)
            # calc loss value
            loss = loss_function(outputs, labels)
            # sum loss value
            running_loss += loss.item()

            # Calculate correct & total prediction
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels.argmax(1)).sum().item()
            total_predictions += labels.size(0)

            # Update progress bar description with loss
            progress_bar.set_postfix(loss=loss.item())

    # Calculate average validation loss
    average_val_loss = running_loss / len(val_loader.dataset)
    # Calculate validation accuracy
    val_accuracy = correct_predictions / total_predictions
    return average_val_loss, val_accuracy





# This is a training loop for selected Epoch
# each epoch will process all training and validation set, chunked into small batch size data
# then measure the loss & accuracy of training and validation set
NUM_EPOCH = 10      # you can change this value

train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(NUM_EPOCH):
    print(f"Epoch {epoch+1}/{NUM_EPOCH}")

    train_loss, train_accuracy = train(model, train_loader, optimizer, loss_function)
    val_loss, val_accuracy = validate(model, validation_loader, loss_function)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accuracies.append(train_accuracy * 100)  # convert to percentage
    val_accuracies.append(val_accuracy * 100)  # convert to percentage

    print(f"Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Train Accuracy = {train_accuracy:.4f}, Val Accuracy = {val_accuracy:.4f}\n")

- Plot Loss and Accuracy of Training vs Validation Set 

In [ ]:
# visualize Loss & Accuracy
import matplotlib.pyplot as plt

epochs = list(range(1, NUM_EPOCH + 1))

# Plotting loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, 'b', label='Training Loss')
plt.plot(epochs, val_losses, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plotting accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracies, 'b', label='Training Accuracy')
plt.plot(epochs, val_accuracies, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.tight_layout()


- Evaluate Model, find Precision, Recal each class data, measure accuracy and compute confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import numpy as np

# define evaluate function for test set
def evaluate(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []

    # Add progress bar for validation loop
    progress_bar = tqdm(test_loader, desc='Evaluating', leave=False)

    with torch.no_grad():
        # iterate over all batched test set
        for inputs, labels in progress_bar:
            inputs = inputs.to(device) # move inputs to device
            labels = labels.to(device) # move labels to device

            # pass input tensor to model
            outputs = model(inputs)
            # get prediction
            _, preds = torch.max(outputs, 1)
            # collect all labels & preds
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    return all_labels, all_preds

# Evaluation on test set
all_labels, all_preds = evaluate(model, test_loader)
all_labels = np.argmax(all_labels, axis=1)

# Calculate classification report
labels = [str(i) for i in range(DATASET_NUM_CLASS)]
print(classification_report(all_labels, all_preds, target_names=labels))

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
plt.xlabel('Predicted Class')
plt.ylabel('Actual Class')
plt.title('Confusion Matrix')
plt.show()

- Download Model 

In [ ]:
# Save the model
torch.save(model.state_dict(), 'trained_model.pt')

# Download the model file
from google.colab import files
files.download('trained_model.pt')

>
>## Discussion
>- Is adding more layer can improve classification performance?
>- Try changing Activation Function to <font color="cyan">ReLU</font>. 
>   - Go to [model definition](#scrollTo=N8cHrdYFi1hL), try changing the hidden layer activation function from <font color="orange">Sigmoid</font> into <font color="orange">ReLU</font><br>
>       <img src="resource/NN_05.png" width="350px"><br><br>
>       - ReLU Activation Function in pytorch
>            ```
>            nn.ReLU()
>            ```
>       - Re-run the training process and compare the performance result
>       - Is changing activation function give any impact?
>
>.

### 📝📝📝 Special care for Deep Networks
- As you add layers, neural networks have more and more <font color="orange">difficulties to converge</font>.
- But why the start-the-art algorithm like YOLO with a lot of layers in it, has a good performance?

##### 1. RELU activation
- One of the answers is <font color="orange">ReLU (Rectified Linear Unit)</font>,<br>
<img src="resource/RELU.png" width="500px"><br><br>
- The <font color="orange">sigmoid activation function</font> is actually quite <font color="cyan">problematic</font> in deep networks.
- It squashes all values between 0 and 1 and when you do so repeatedly, neuron outputs and their <font color="orange">gradients can vanish entirely</font>.
- It was mentioned for historical reasons, but modern networks use the <font color="orange">RELU (Rectified Linear Unit)</font>.<br>
<img src="resource/NN_05.png" width="350px"><br>
- RELU was shown to have better convergence properties almost everywhere.
    - introduce non-linearity with just simple function
    - making network sparse and efficient by sets any negative value to zero
    - computationally cheaper compared to sigmoid or tanh
    - maintains strong gradients during training, help deeper networks to converge effectively

##### 2. Better Optimizer
- <font color="orange">Optimizer</font> help to minimizing the loss function by updating the neural network weights.
- Those minimum value in parameter space is called <font color="orange">'local minima'</font>,
- <font color="orange">SGD Optimizer</font> might not perform well on <font color="orange">very high-dimensional spaces</font> like here — we have on the order of 10K weights and biases.
- <font color="orange">"saddle points"</font> are frequent on that kind of network. 
- These are points that are not local minima, but where the <font color="orange">gradient is zero</font> and the <font color="orange">SGD optimizer</font> stays stuck there. 
- Open <font color="orange">'2.2 neural_network_optimizer.ipynb'</font> in Google Colab to learn more...<br> 
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-Image-Classification/blob/main/Pertemuan%202/2.2%20neural_network_optimizer.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br><br>
<img src="resource/LocalMinima.png" width="700px">

_________________________________________________________________________
<br><br><br>
# Source
- https://medium.com/analytics-vidhya/journey-of-gradient-descent-from-local-to-global-c851eba3d367
- https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist#6